# Scraping posts from r/conspiracy

Extraction of all the posts resent in r/conspiracy, with title, content and url.
First of all we need to import praw and access reddit:

In [1]:
import praw
reddit = praw.Reddit(client_id = "", client_secret= "", user_agent= "")

We can now access the subreddit, extract all the posts in it adn check which other subreddit reposted that posts and how much:

In [2]:
import pandas as pd
post_list=list()
subreddit_list = list()
conspiracy_dict=dict()

#put the subreddit already passed here
id_to_avoid = ["conspiracy"]

for i in reddit.subreddit("conspiracy").top(limit=5000):
    post_list.append((i.title, i.score, "https://www.reddit.com"+i.permalink))


for post in post_list:
    for repost in reddit.subreddit('all').search('url:'+post[2]):
        if repost.subreddit_id not in id_to_avoid:
            subreddit_url = str(repost.subreddit)
            subreddit_url = "https://www.reddit.com/r/" + subreddit_url
            if subreddit_url in conspiracy_dict.keys():
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                conspiracy_dict[subreddit_url][1][0] +=1
            else:
                conspiracy_dict[subreddit_url]=[[], []]
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                conspiracy_dict[subreddit_url][1].append(1)


df = pd.DataFrame(conspiracy_dict)
df.to_csv(r'results/conspiracy_top.csv',index=False)


